In [1]:
import torch
import torch.nn as nn
from torchquad import VEGAS 
import numpy as np 

20:43:13|TQ-INFO| Initializing torchquad.


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
        nn.Linear(1,10),
        nn.Sigmoid(),
        nn.Linear(10,1, bias=False)
        )
        
    def forward(self, x):
        N = self.layers(x)
        return N

In [3]:
model = NeuralNetwork()
upper_r = 6
lower_r = 1e-2
steps = 100
R_train = torch.Tensor(np.linspace(lower_r, upper_r, steps)[:,None])
beta = 2
global Z
Z = 1
global e
#e = -1.602e-19
e = -1
global hbar
#hbar = 1.054e-34
hbar = 1
global m
#m = 9.109e-31
m = 1
global l
l = 0
V = lambda r: -(Z*e**2)/r
Phi_t = lambda r: torch.exp(-beta*r**2) * model.forward(r)

In [4]:
vg = VEGAS()
integral_value = vg.integrate(Phi_t, dim=1, N=200, integration_domain=[[-6,6]])
integral_value

/home/andresgo/anaconda3/lib/python3.9/site-packages/torchquad/integration/vegas_stratification.py:113: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = tmp // self.N_strat
20:43:13|TQ-INFO| Iteration 1, Acc=nan, Result=-3.4758e-02,neval=30
20:43:13|TQ-INFO| Iteration 2, Acc=1.9710e-01, Result=4.7876e-02,neval=64
20:43:13|TQ-INFO| Iteration 3, Acc=1.6517e-01, Result=1.3952e-02,neval=94
20:43:13|TQ-INFO| Iteration 4, Acc=1.7719e-01, Result=1.7693e-02,neval=128
20:43:13|TQ-INFO| Iteration 5, Acc=8.0452e-02, Result=7.4551e-02,neval=162
20:43:13|TQ-INFO| Iteration 6, Acc=5.2784e-02, Result=5.2946e-02,neval=216
20:43:13|TQ-INFO| Computed integral after 216 evals w

tensor(0.0529, grad_fn=<DivBackward0>)

In [5]:
f = lambda x: -x**2
integral_value1 = vg.integrate(f, dim=1, N=100000, integration_domain=[[0,1]])
integral_value1

20:43:13|TQ-INFO| Iteration 1, Acc=nan, Result=-3.3334e-01,neval=15000
20:43:13|TQ-INFO| Iteration 2, Acc=nan, Result=-3.3333e-01,neval=25682
20:43:13|TQ-INFO| Iteration 3, Acc=nan, Result=-3.3333e-01,neval=34950
20:43:14|TQ-INFO| Iteration 4, Acc=nan, Result=-3.3334e-01,neval=44406
20:43:14|TQ-INFO| Iteration 5, Acc=nan, Result=-3.3335e-01,neval=53784
20:43:14|TQ-INFO| Computed integral after 53784 evals was -3.33335161e-01.


tensor(-0.3333)

In [6]:
integral_value2 = vg.integrate(lambda r: Phi_t(r)**2, dim=1, N=100000, integration_domain=[[0,1]])
integral_value2

20:43:14|TQ-INFO| Iteration 1, Acc=7.4054e-07, Result=3.0802e-04,neval=15000
20:43:14|TQ-INFO| Iteration 2, Acc=4.8778e-07, Result=3.0804e-04,neval=26560
20:43:14|TQ-INFO| Iteration 3, Acc=1.3872e-06, Result=3.0807e-04,neval=35728
20:43:14|TQ-INFO| Iteration 4, Acc=1.4192e-06, Result=3.0800e-04,neval=44944
20:43:15|TQ-INFO| Iteration 5, Acc=2.0375e-06, Result=3.0800e-04,neval=54150
20:43:15|TQ-INFO| Iteration 6, Acc=2.0292e-06, Result=3.0796e-04,neval=73264
20:43:15|TQ-INFO| Iteration 7, Acc=2.2292e-06, Result=3.0800e-04,neval=92374
20:43:15|TQ-INFO| Computed integral after 92374 evals was 3.07975773e-04.


tensor(0.0003, grad_fn=<DivBackward0>)

In [7]:
import vegas
import math

def f(x):
    dx2 = 0
    for d in range(4):
        dx2 += (x[d] - 0.5) ** 2
    return math.exp(-dx2 * 100.) * 1013.2118364296088

integ = vegas.Integrator([[-1, 1], [0, 1], [0, 1], [0, 1]])

result = integ(f, nitn=10, neval=1000)
print(result.summary())
print('result = %s    Q = %.2f' % (result, result.Q))

itn   integral        wgt average     chi2/dof        Q
-------------------------------------------------------
  1   0.34(12)        0.34(12)            0.00     1.00
  2   0.94(12)        0.650(86)          12.41     0.00
  3   0.901(65)       0.810(52)           8.90     0.00
  4   0.917(54)       0.862(37)           6.63     0.00
  5   0.954(34)       0.913(25)           5.82     0.00
  6   1.014(25)       0.963(18)           6.27     0.00
  7   0.994(18)       0.979(13)           5.48     0.00
  8   1.021(16)       0.9954(98)          5.35     0.00
  9   0.971(13)       0.9869(79)          4.96     0.00
 10   1.012(11)       0.9951(65)          4.78     0.00

result = 0.9951(65)    Q = 0.00


In [8]:
import torch.nn as nn
import torch
import numpy as np
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.Sequential(
        nn.Linear(1,10),
        nn.Sigmoid(),
        nn.Linear(10,1, bias=False)
        )
        
    def forward(self, x):
        N = self.layers(x)
        return N

In [9]:
model = NeuralNetwork()

V = lambda r: -(Z*e**2)/r

def Phi_t(r):
    r = torch.Tensor(r)
    return np.exp(-2*r**2) * model.forward(r).detach().numpy()

In [10]:
integ = vegas.Integrator([[-6,6]])
result = integ(Phi_t, nitn=10, neval=1000)
result

RAvgArray([0.45272(12)], dtype=object)

In [11]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Linear)):
            nn.init.kaiming_normal_(m.weight.data)

model = NeuralNetwork()
initialize_weights(model)
result = integ(Phi_t, nitn=10, neval=200)
print(result)

[-1.7459(75)]


In [12]:
result = integ(Phi_t, nitn=10, neval=200)
print(np.squeeze(result))

-1.7461(52)


In [13]:
result.itn_results

[array([-1.783(38)], dtype=object),
 array([-1.771(16)], dtype=object),
 array([-1.868(82)], dtype=object),
 array([-1.762(16)], dtype=object),
 array([-1.762(15)], dtype=object),
 array([-1.744(13)], dtype=object),
 array([-1.738(12)], dtype=object),
 array([-1.724(10)], dtype=object),
 array([-1.757(45)], dtype=object),
 array([-1.809(55)], dtype=object)]

In [14]:
str(np.squeeze(result)).rstrip('()')

'-1.7461(52'

In [15]:
result = integ(Phi_t, nitn=10, neval=200)
np.squeeze(result).flatten()

RAvgArray([-1.7753(62)], dtype=object)

In [16]:
import re

n = str(np.squeeze(result))
n

'-1.7753(62)'

In [17]:
m = re.search("\d\S\d*", n)
m

<re.Match object; span=(1, 7), match='1.7753'>

In [18]:
type(re.search("\d\S\d*", str(np.squeeze(result)))[0])

str

In [19]:
np.array(float(re.search("\d\S\d*", str(np.squeeze(result)))[0]))

array(1.7753)

In [20]:
type(np.array(float(re.search("\d\S\d*", str(np.squeeze(result)))[0])))

numpy.ndarray

In [21]:
torch.Tensor(np.array(float(re.search("\d\S\d*", str(np.squeeze(result)))[0])))

tensor(1.7753)

In [22]:
float(re.search("\d\S\d*", str(np.squeeze(result)))[0])

1.7753

In [23]:
type(torch.Tensor(np.array(float(re.search("\d\S\d*", str(np.squeeze(result)))[0]))))

torch.Tensor